In [ ]:
!pip install spacy
!python -m spacy download es_core_news_sm

In [ ]:
!git clone https://github.com/MadisonAcosta/PyTutor.git


In [ ]:
import json  # Importa la biblioteca JSON para manejar archivos en este formato
from difflib import SequenceMatcher
from collections import deque  # Importa deque para manejar una cola de memoria limitada
import random  # Importa random para seleccionar respuestas aleatorias

def cargar_json(nombre_archivo):
    try:
        with open(nombre_archivo, "r", encoding="utf-8") as file:
            return json.load(file)
    except FileNotFoundError:
        print(f"Error: No se encontró el archivo '{nombre_archivo}'. Verifica su ubicación.")
        exit(1)  # Termina el programa con código de error
    except json.JSONDecodeError:
        print(f"Error: El archivo '{nombre_archivo}' tiene un formato incorrecto.")
        exit(1)

# Cargar intents y datos de celulares con manejo de errores
intents = cargar_json("intents.json")
celulares = cargar_json("celulares_con_benchmarks.json")

# Inicializar la memoria del chatbot con un tamaño máximo de 5 mensajes
memoria_chat = deque(maxlen=5)
preferencias_usuario = {}  # Diccionario para almacenar las preferencias del usuario
inicio_recoleccion = False  # Bandera para indicar si se están recopilando preferencias

# Extraer preguntas del flujo de evaluaciones en intents.json
preguntas_flujo = intents["evaluacionesChat"][0]["responses"]
indice_pregunta = 0  # Índice para rastrear en qué pregunta del flujo está el chatbot

# Extraer patrones y respuestas de saludos del archivo intents.json
saludos_patterns = {pat.lower() for intent in intents["intents"] if intent["tag"] == "saludo" for pat in intent["patterns"]}
saludos_responses = [resp for intent in intents["intents"] if intent["tag"] == "saludo" for resp in intent["responses"]]

def generar_respuesta_recomendacion(celular):
    respuesta = f"Te recomiendo el {celular['Model Name']} de {celular['Company Name']}.\n"
    respuesta += f"- 📆 Año de lanzamiento: {celular['Launched Year']}\n"
    respuesta += f"- 💰 Precio: ${celular['Launched Price (USA)']:.2f}\n"
    respuesta += f"- 🔋 Batería: {celular['Battery Capacity (mAh)']} mAh\n"
    respuesta += f"- 📷 Cámara frontal: {celular['Front Camera (MP)']} MP\n"
    respuesta += f"- 📱 Tamaño de pantalla: {celular['Screen Size (inches)']} pulgadas\n"
    respuesta += f"- ⚙️ Procesador: {celular['Processor']} ({celular['cores']} núcleos a {celular['clock']/1000:.2f} GHz)\n"
    return respuesta

# Función para validar si la entrada del usuario está vacía
def validar_entrada_vacia(user_input):
    if not user_input:
        print("Chatbot: No entendí tu respuesta. Por favor, escribe algo.")
        return False
    return True

# Función para validar si el usuario quiere salir
salida_patterns = {"salir", "adiós", "hasta luego", "me voy", "nos vemos"}

def validar_salida(user_input):
    """Verifica si el usuario quiere salir usando diferentes palabras clave."""
    user_input = normalizar_entrada(user_input)
    if user_input in salida_patterns:
        print("Chatbot: ¡Hasta luego! 😊")
        return True
    return False

# Función para validar si la entrada del usuario es un saludo
def validar_saludo(user_input):
    if user_input in saludos_patterns:
        print(f"Chatbot: {random.choice(saludos_responses)}")  # Responde con un saludo aleatorio
        return True
    return False

# Función para validar si el usuario ingresó un número
def validar_numero(user_input):
    try:
        return float(user_input) if "." in user_input else int(user_input)
    except ValueError:
        print("Chatbot: Por favor, ingresa un número válido.")
        return None

# Función para validar respuestas de sí o no
# Extraer respuestas positivas y negativas desde intents.json
respuestas_positivas = {pat.lower() for intent in intents["evaluacionesChat"] if intent["tag"] == "respuestas_positivas" for pat in intent["responses"]}
respuestas_negativas = {pat.lower() for intent in intents["evaluacionesChat"] if intent["tag"] == "respuestas_negativas" for pat in intent["responses"]}

def validar_respuesta_si_no(user_input):
    user_input = user_input.strip().lower()
    print(f"Validando respuesta: '{user_input}'")  
    if user_input in respuestas_positivas:
        print("Reconocido como positivo")  
        return True
    elif user_input in respuestas_negativas:
        print("Reconocido como negativo")  
        return False
    print("No se reconoce la respuesta")  
    return None

# Función para normalizar la entrada eliminando espacios y convirtiendo a minúsculas
def normalizar_entrada(user_input):
    return user_input.strip().lower()

# Función para validar si la marca ingresada está en la lista de celulares
def extraer_marcas_desde_intents(intents):
    """Extrae las marcas de celulares desde la estructura de intents.json."""
    for intent in intents.get("evaluacionesChat", []):
        if intent.get("tag") == "celulares_por_marca":
            return set(marca.lower() for marca in intent.get("patterns", []))
    return set()  # Si no se encuentra el tag, retorna un conjunto vacío

def validar_marca(user_input, marcas_disponibles):
    user_input = user_input.lower()

    # Si el usuario responde con una respuesta negativa, no asignamos marca y seguimos con el flujo.
    if user_input in respuestas_negativas:
        print("Chatbot: No hay problema, podemos continuar sin elegir una marca específica.")
        return "ninguna"  # Devuelve un valor neutral para que el flujo siga sin marca específica
    
    if user_input not in marcas_disponibles:
        print("Chatbot: No encontré esa marca. Aquí algunas opciones:", ", ".join(marcas_disponibles))
        return None  # Devuelve None para seguir preguntando

    return user_input  # Retorna la marca si es válida



# Extraer marcas desde intents.json
marcas_disponibles = extraer_marcas_desde_intents(intents)

# Función para recomendar un celular basado en las preferencias del usuario
def recomendar_celular():
    celulares_filtrados = celulares  # Lista inicial de celulares
    
    # Filtrado básico según preferencias
    for clave, valor in preferencias_usuario.items():
        if clave == "presupuesto":
            celulares_filtrados = [c for c in celulares_filtrados if c.get("Launched Price (USA)", float("inf")) <= valor]
        elif clave == "camara":
            celulares_filtrados = [c for c in celulares_filtrados if c.get("Front Camera (MP)", 0) >= valor]
        elif clave == "rendimiento":
            celulares_filtrados = [c for c in celulares_filtrados if c.get("RAM (GB)", 0) >= valor]
        elif clave == "bateria":
            celulares_filtrados = [c for c in celulares_filtrados if c.get("Battery Capacity (mAh)", 0) >= valor]
        elif clave == "marca":
            celulares_filtrados = [c for c in celulares_filtrados if valor.lower() in c.get("Company Name", "").lower()]
        elif clave == "cpuScore":
            celulares_filtrados = [c for c in celulares_filtrados if c.get("cpuScore", 0) >= valor]
        elif clave == "gpuScore":
            celulares_filtrados = [c for c in celulares_filtrados if c.get("gpuScore", 0) >= valor]

    if not celulares_filtrados:
        return None

    # Asignar una puntuación basada en múltiples factores
    def calcular_puntuacion(celular):
        puntuacion = 0
        if "presupuesto" in preferencias_usuario:
            diferencia_precio = abs(preferencias_usuario["presupuesto"] - celular.get("Launched Price (USA)", float("inf")))
            puntuacion -= diferencia_precio  # Penalización por alejamiento del presupuesto
        if "camara" in preferencias_usuario:
            puntuacion += celular.get("Front Camera (MP)", 0) * 2  # Mayor peso a la cámara
        if "rendimiento" in preferencias_usuario:
            puntuacion += celular.get("RAM (GB)", 0) * 3
        if "bateria" in preferencias_usuario:
            puntuacion += celular.get("Battery Capacity (mAh)", 0) * 0.5
        if "cpuScore" in preferencias_usuario:
            puntuacion += celular.get("cpuScore", 0)
        if "gpuScore" in preferencias_usuario:
            puntuacion += celular.get("gpuScore", 0)
        return puntuacion

    # Ordenamos los celulares por puntuación
    celulares_ordenados = sorted(celulares_filtrados, key=calcular_puntuacion, reverse=True)

    return celulares_ordenados[0] if celulares_ordenados else None

# Función principal del chatbot
def chatbot():
    global inicio_recoleccion, preferencias_usuario, indice_pregunta
    print("Hola, ¿en qué puedo ayudarte?")
    while True:
        user_input = normalizar_entrada(input("Tú: ")).lower()  # Convertimos la entrada a minúsculas
        memoria_chat.append(user_input)  # Agrega la entrada a la memoria del chatbot

        if not validar_entrada_vacia(user_input):
            continue
        
        if validar_salida(user_input):
            break
        
        if validar_saludo(user_input):
            continue
        
        if not inicio_recoleccion:
            if any(word in user_input for word in ["recomendar", "celular", "jugar", "busco"]):
                print("Chatbot: Claro, dime tus preferencias.")
                inicio_recoleccion = True
                indice_pregunta = 0
                preferencias_usuario.clear()
            else:
                print("Chatbot: Lo siento, no entendí tu pregunta. ¿Puedes reformularla?")
            continue

        # 💡 Aquí volvemos a validar la marca antes de seguir con las preguntas
        if any(marca in user_input for marca in marcas_disponibles):
            marca_validada = validar_marca(user_input, marcas_disponibles)
            if marca_validada:
                if marca_validada != "ninguna":  # Solo guardamos si el usuario eligió una marca
                    preferencias_usuario["marca"] = marca_validada
                    print(f"Chatbot: Has seleccionado la marca '{marca_validada}'. ¿Tienes alguna otra preferencia?")
                else:
                    print("Chatbot: Continuemos con las demás preguntas.")

                indice_pregunta += 1  # 🚀 Avanzamos a la siguiente pregunta
            continue

        if indice_pregunta < len(preguntas_flujo):
            pregunta_actual = preguntas_flujo[indice_pregunta]["pregunta"]
            respuesta_validada = None  # Inicializamos la respuesta

            if "batería" in pregunta_actual.lower():
                respuesta_validada = validar_respuesta_si_no(user_input)
                if respuesta_validada:
                    preferencias_usuario["bateria"] = 4000
                indice_pregunta += 1
            elif "presupuesto" in pregunta_actual.lower():
                respuesta_validada = validar_numero(user_input)
                if respuesta_validada is not None:
                    preferencias_usuario["presupuesto"] = respuesta_validada
                    indice_pregunta += 1
            elif "rendimiento" in pregunta_actual.lower():
                respuesta_validada = validar_respuesta_si_no(user_input)
                if respuesta_validada:  # Si responde "sí"
                    preferencias_usuario["cpuScore"] = 2500
                    indice_pregunta += 1
            elif "gráficos" in pregunta_actual.lower():
                respuesta_validada = validar_respuesta_si_no(user_input)
                if respuesta_validada:  # Si responde "sí"
                    preferencias_usuario["gpuScore"] = 5000
                    indice_pregunta += 1

            # Solo avanzamos a la siguiente pregunta si la respuesta es válida
            if respuesta_validada is not None:
                indice_pregunta += 1
            else:
                print("Chatbot: No entendí tu respuesta. ¿Podrías intentarlo de nuevo?")

            if indice_pregunta < len(preguntas_flujo):
                print(f"Chatbot: {preguntas_flujo[indice_pregunta]['pregunta']}")
            else:
                celular_recomendado = recomendar_celular()

                if celular_recomendado:
                    print("Chatbot:", generar_respuesta_recomendacion(celular_recomendado))
                else:
                    print("Chatbot: No encontré un celular que cumpla con todas tus preferencias. ¿Quieres que busque opciones cercanas? (sí/no)")

                    respuesta = normalizar_entrada(input("Tú: ")).lower()  # Convertimos la respuesta a minúsculas

                    if validar_respuesta_si_no(respuesta) is True:
                        print("Chatbot: Voy a relajar algunas restricciones para encontrar más opciones...")

                        # Crear una copia de las preferencias originales
                        preferencias_relajadas = preferencias_usuario.copy()

                        # Relajar algunas restricciones gradualmente
                        if "presupuesto" in preferencias_relajadas:
                            preferencias_relajadas["presupuesto"] *= 1.2  # Aumentar un 20% el presupuesto
                        if "bateria" in preferencias_relajadas:
                            preferencias_relajadas["bateria"] -= 500  # Reducir la exigencia de batería
                        if "cpuScore" in preferencias_relajadas:
                            preferencias_relajadas["cpuScore"] *= 0.8  # Permitir CPU menos potentes
                        if "gpuScore" in preferencias_relajadas:
                            preferencias_relajadas["gpuScore"] *= 0.8  # Permitir GPU menos potentes

                        # Intentar recomendar con las nuevas preferencias relajadas
                        celular_alternativo = recomendar_celular()

                        if celular_alternativo:
                            print("Chatbot: No encontré exactamente lo que buscabas, pero este podría interesarte:")
                            print(generar_respuesta_recomendacion(celular_alternativo))
                        else:
                            print("Chatbot: Aún con las restricciones relajadas, no encontré una opción adecuada. Tal vez quieras intentarlo con otros criterios.")
                    
                inicio_recoleccion = False
                indice_pregunta = 0
                preferencias_usuario.clear()
chatbot()